In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    X = data.drop(columns=['Eligible'])
    y = data['Eligible']
    return X, y

def train_ridge_model(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    ridge_model = Ridge()
    ridge_model.fit(X_train_scaled, y_train)
    return ridge_model, scaler

# Load and train on VehicleTrainingDataset.csv
X_train, y_train = load_and_preprocess_data('VehicleTrainingDataset_Noisy_0.01.csv')
ridge_model, scaler = train_ridge_model(X_train, y_train)

# Predict eligibility scores on 1000VehicleDataset.csv
vehicles_df = pd.read_csv('1000VehicleDataset_Noisy_0.01.csv')
X_test = vehicles_df.drop(columns=['Eligible'])
X_test_scaled = scaler.transform(X_test)
predicted_scores = ridge_model.predict(X_test_scaled)

# Assuming you have access to actual scores, replace this line with the actual score loading logic if available
y_actual = vehicles_df['Eligible']  # This would be prior to overwriting with predictions if you run this block again

# Replace actual scores with predicted ones
vehicles_df['Eligible'] = predicted_scores  

# Calculate metrics
mae = mean_absolute_error(y_actual, predicted_scores)
rmse = np.sqrt(mean_squared_error(y_actual, predicted_scores))
r_squared = r2_score(y_actual, predicted_scores)
rae = np.sum(np.abs(y_actual - predicted_scores)) / np.sum(np.abs(y_actual - np.mean(y_actual)))

# Output the results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")
print(f"RAE: {rae}")


MAE: 0.45497986053091444
RMSE: 0.5737672522389483
R-squared: 0.997773370716979
RAE: 0.04751820477894557


In [2]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-------- Rollout Summary --------
Total mean reward: -98.0
Standard deviation of reward: 0.0
Average successful assignments: 2.6
All assignments history: [2, 5, 7, 4, 6, 4, 5, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -91.6    |
| time/              |          |
|    fps             | 664      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -92.0
Standard deviation of reward: 0.0
Average successful assignments: 3.55
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -90.8       |
| time/                   |             |
|    fps                  | 604         |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.006986077 |
|    clip_fraction        | 0.0797      |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.02       |
|    explained_variance   | -0.37       |
|    learning_rate        | 0.00018     |
|    loss                 | 2.09        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0331     |
|    value_loss           | 14.9        |
--

-------- Rollout Summary --------
Total mean reward: 18.0
Standard deviation of reward: 0.0
Average successful assignments: 20.495
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -89.6        |
| time/                   |              |
|    fps                  | 573          |
|    iterations           | 10           |
|    time_elapsed         | 17           |
|    total_timesteps      | 10240        |
| train/                  |              |
|    approx_kl            | 0.0074089253 |
|    clip_fraction        | 0.113        |
|    clip_range           | 0.15         |
|    entropy_loss         | -4.96        |
|    explained_variance   | 0.276        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.93         |
|    n_updates            | 90           |
|    policy_gradient_loss | -0.0371      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 47.52045454545455
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 23.6         |
| time/                   |              |
|    fps                  | 548          |
|    iterations           | 66           |
|    time_elapsed         | 123          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0039434694 |
|    clip_fraction        | 0.0831       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.92        |
|    explained_variance   | 0.925        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.507        |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0261      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 49.73783783783784
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 29.5        |
| time/                   |             |
|    fps                  | 547         |
|    iterations           | 74          |
|    time_elapsed         | 138         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.005200545 |
|    clip_fraction        | 0.088       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.89       |
|    explained_variance   | 0.94        |
|    learning_rate        | 0.00018     |
|    loss                 | 1.12        |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0228     |
|    value_loss           | 1.02

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 51.44634146341463
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 29.4         |
| time/                   |              |
|    fps                  | 547          |
|    iterations           | 82           |
|    time_elapsed         | 153          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0055013793 |
|    clip_fraction        | 0.104        |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.97        |
|    explained_variance   | 0.929        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.373        |
|    n_updates            | 810          |
|    policy_gradient_loss | -0.0246      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 52.88
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 30.1        |
| time/                   |             |
|    fps                  | 544         |
|    iterations           | 90          |
|    time_elapsed         | 169         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.004479411 |
|    clip_fraction        | 0.0875      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.92       |
|    explained_variance   | 0.945       |
|    learning_rate        | 0.00018     |
|    loss                 | 1.06        |
|    n_updates            | 890         |
|    policy_gradient_loss | -0.0229     |
|    value_loss           | 0.958       |
--

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 54.093877551020405
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 31.2        |
| time/                   |             |
|    fps                  | 534         |
|    iterations           | 98          |
|    time_elapsed         | 187         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.009457154 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.98       |
|    explained_variance   | 0.942       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.184       |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0284     |
|    value_loss           | 1.0

In [3]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -96.0
Standard deviation of reward: 0.0
Average successful assignments: 2.75
All assignments history: [1, 2, 3, 8, 3, 2, 6, 2, 3, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -93      |
| time/              |          |
|    fps             | 551      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -96.0
Standard deviation of reward: 0.0
Average successful assignments: 2.775
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -92.9        |
| time/                   |              |
|    fps                  | 520          |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0060715135 |
|    clip_fraction        | 0.0592       |
|    clip_range           | 0.15         |
|    entropy_loss         | -5.02        |
|    explained_variance   | -0.315       |
|    learning_rate        | 0.00018      |
|    loss                 | 2.41         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0312      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 38.0
Standard deviation of reward: 0.0
Average successful assignments: 48.26969696969697
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 20.2        |
| time/                   |             |
|    fps                  | 357         |
|    iterations           | 66          |
|    time_elapsed         | 188         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.003874706 |
|    clip_fraction        | 0.0728      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.22       |
|    explained_variance   | 0.871       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.841       |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.0252     |
|    value_loss           | 2.08

-------- Rollout Summary --------
Total mean reward: 38.0
Standard deviation of reward: 0.0
Average successful assignments: 50.054054054054056
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 21.3        |
| time/                   |             |
|    fps                  | 358         |
|    iterations           | 74          |
|    time_elapsed         | 211         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.004437351 |
|    clip_fraction        | 0.0999      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.14       |
|    explained_variance   | 0.892       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.539       |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0271     |
|    value_loss           | 1.8

-------- Rollout Summary --------
Total mean reward: 38.0
Standard deviation of reward: 0.0
Average successful assignments: 51.58170731707317
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 24.1        |
| time/                   |             |
|    fps                  | 359         |
|    iterations           | 82          |
|    time_elapsed         | 233         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.004325694 |
|    clip_fraction        | 0.0827      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.1        |
|    explained_variance   | 0.898       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.568       |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.0246     |
|    value_loss           | 1.75

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 52.88611111111111
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 26.1         |
| time/                   |              |
|    fps                  | 359          |
|    iterations           | 90           |
|    time_elapsed         | 256          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0046202336 |
|    clip_fraction        | 0.0943       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.04        |
|    explained_variance   | 0.922        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.384        |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0273      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 54.03775510204082
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 26.9         |
| time/                   |              |
|    fps                  | 360          |
|    iterations           | 98           |
|    time_elapsed         | 278          |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0042031407 |
|    clip_fraction        | 0.086        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.08        |
|    explained_variance   | 0.931        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.193        |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.0256      |
|    value_lo

In [4]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -88.0
Standard deviation of reward: 0.0
Average successful assignments: 4.9
All assignments history: [4, 6, 2, 0, 4, 6, 4, 5, 4, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -92.4    |
| time/              |          |
|    fps             | 435      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -90.0
Standard deviation of reward: 0.0
Average successful assignments: 4.875
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -91.5       |
| time/                   |             |
|    fps                  | 398         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.006746405 |
|    clip_fraction        | 0.0619      |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.02       |
|    explained_variance   | -0.301      |
|    learning_rate        | 0.00018     |
|    loss                 | 3.4         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.033      |
|    value_loss           | 18.7        |
-

...

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 46.92272727272727
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 20.3         |
| time/                   |              |
|    fps                  | 366          |
|    iterations           | 66           |
|    time_elapsed         | 184          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0043616667 |
|    clip_fraction        | 0.0787       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.06        |
|    explained_variance   | 0.895        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.601        |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0215      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 49.13310810810811
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 26           |
| time/                   |              |
|    fps                  | 363          |
|    iterations           | 74           |
|    time_elapsed         | 208          |
|    total_timesteps      | 75776        |
| train/                  |              |
|    approx_kl            | 0.0046556983 |
|    clip_fraction        | 0.0902       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.92        |
|    explained_variance   | 0.917        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.486        |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.024       |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 51.014634146341464
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 29.5         |
| time/                   |              |
|    fps                  | 363          |
|    iterations           | 82           |
|    time_elapsed         | 231          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0043614395 |
|    clip_fraction        | 0.092        |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.86        |
|    explained_variance   | 0.925        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.363        |
|    n_updates            | 810          |
|    policy_gradient_loss | -0.0254      |
|    value_l

-------- Rollout Summary --------
Total mean reward: 46.0
Standard deviation of reward: 0.0
Average successful assignments: 52.67888888888889
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 32.3         |
| time/                   |              |
|    fps                  | 358          |
|    iterations           | 90           |
|    time_elapsed         | 257          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0043445453 |
|    clip_fraction        | 0.0717       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.88        |
|    explained_variance   | 0.929        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.31         |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0208      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 46.0
Standard deviation of reward: 0.0
Average successful assignments: 54.10408163265306
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 34.2        |
| time/                   |             |
|    fps                  | 359         |
|    iterations           | 98          |
|    time_elapsed         | 279         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.007152699 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.87       |
|    explained_variance   | 0.916       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.351       |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0261     |
|    value_loss           | 1.16

In [5]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -96.0
Standard deviation of reward: 0.0
Average successful assignments: 3.1
All assignments history: [5, 3, 3, 4, 3, 5, 3, 3, 7, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -91.6    |
| time/              |          |
|    fps             | 366      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -66.0
Standard deviation of reward: 0.0
Average successful assignments: 6.925
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -91.3       |
| time/                   |             |
|    fps                  | 372         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.007508908 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.02       |
|    explained_variance   | 0.067       |
|    learning_rate        | 0.00018     |
|    loss                 | 1.69        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0375     |
|    value_loss           | 13.4        |
-

...

-------- Rollout Summary --------
Total mean reward: 36.0
Standard deviation of reward: 0.0
Average successful assignments: 45.6
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 18          |
| time/                   |             |
|    fps                  | 292         |
|    iterations           | 66          |
|    time_elapsed         | 230         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.003364246 |
|    clip_fraction        | 0.0827      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.14       |
|    explained_variance   | 0.883       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.666       |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.0251     |
|    value_loss           | 1.89        |
---

-------- Rollout Summary --------
Total mean reward: 38.0
Standard deviation of reward: 0.0
Average successful assignments: 47.7
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 22.3        |
| time/                   |             |
|    fps                  | 284         |
|    iterations           | 74          |
|    time_elapsed         | 266         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.003845436 |
|    clip_fraction        | 0.0844      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.05       |
|    explained_variance   | 0.921       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.47        |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0232     |
|    value_loss           | 1.49        |
---

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 49.51341463414634
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 25          |
| time/                   |             |
|    fps                  | 277         |
|    iterations           | 82          |
|    time_elapsed         | 302         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.005040629 |
|    clip_fraction        | 0.0872      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.88       |
|    explained_variance   | 0.943       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.24        |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.0253     |
|    value_loss           | 1.05

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 51.06666666666667
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 27.7        |
| time/                   |             |
|    fps                  | 271         |
|    iterations           | 90          |
|    time_elapsed         | 338         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.004259509 |
|    clip_fraction        | 0.0936      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.92       |
|    explained_variance   | 0.93        |
|    learning_rate        | 0.00018     |
|    loss                 | 0.179       |
|    n_updates            | 890         |
|    policy_gradient_loss | -0.027      |
|    value_loss           | 1.24

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 52.429591836734694
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 29.8         |
| time/                   |              |
|    fps                  | 265          |
|    iterations           | 98           |
|    time_elapsed         | 378          |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0047206995 |
|    clip_fraction        | 0.0988       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.86        |
|    explained_variance   | 0.95         |
|    learning_rate        | 0.00018      |
|    loss                 | 0.12         |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.0255      |
|    value_l

In [6]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -82.0
Standard deviation of reward: 0.0
Average successful assignments: 6.15
All assignments history: [2, 5, 4, 5, 6, 2, 2, 1, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -93.4    |
| time/              |          |
|    fps             | 237      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -62.0
Standard deviation of reward: 0.0
Average successful assignments: 8.775
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -92.9        |
| time/                   |              |
|    fps                  | 220          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0061978074 |
|    clip_fraction        | 0.0599       |
|    clip_range           | 0.15         |
|    entropy_loss         | -5.02        |
|    explained_variance   | -0.136       |
|    learning_rate        | 0.00018      |
|    loss                 | 2.55         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0315      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 45.696969696969695
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 17.7         |
| time/                   |              |
|    fps                  | 211          |
|    iterations           | 66           |
|    time_elapsed         | 319          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0058189696 |
|    clip_fraction        | 0.101        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.24        |
|    explained_variance   | 0.866        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.633        |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0255      |
|    value_l

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 47.91148648648649
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 22.4         |
| time/                   |              |
|    fps                  | 212          |
|    iterations           | 74           |
|    time_elapsed         | 356          |
|    total_timesteps      | 75776        |
| train/                  |              |
|    approx_kl            | 0.0046209125 |
|    clip_fraction        | 0.0818       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.11        |
|    explained_variance   | 0.912        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.366        |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.0233      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 49.78414634146341
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 26.2        |
| time/                   |             |
|    fps                  | 215         |
|    iterations           | 82          |
|    time_elapsed         | 389         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.004738368 |
|    clip_fraction        | 0.0833      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.04       |
|    explained_variance   | 0.895       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.409       |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.0267     |
|    value_loss           | 1.65

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 51.37277777777778
All assignments history: []
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | 28         |
| time/                   |            |
|    fps                  | 218        |
|    iterations           | 90         |
|    time_elapsed         | 422        |
|    total_timesteps      | 92160      |
| train/                  |            |
|    approx_kl            | 0.00519994 |
|    clip_fraction        | 0.108      |
|    clip_range           | 0.15       |
|    entropy_loss         | -2.02      |
|    explained_variance   | 0.91       |
|    learning_rate        | 0.00018    |
|    loss                 | 0.425      |
|    n_updates            | 890        |
|    policy_gradient_loss | -0.0284    |
|    value_loss           | 1.36       |
----------

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 52.71632653061224
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 29.2        |
| time/                   |             |
|    fps                  | 220         |
|    iterations           | 98          |
|    time_elapsed         | 455         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.005021119 |
|    clip_fraction        | 0.0968      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.97       |
|    explained_variance   | 0.931       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.21        |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0248     |
|    value_loss           | 0.97

In [7]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -100.0
Standard deviation of reward: 0.0
Average successful assignments: 2.15
All assignments history: [4, 3, 6, 4, 6, 2, 5, 6, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -91.4    |
| time/              |          |
|    fps             | 304      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -74.0
Standard deviation of reward: 0.0
Average successful assignments: 5.4
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -91.4       |
| time/                   |             |
|    fps                  | 278         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.006848591 |
|    clip_fraction        | 0.0665      |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.02       |
|    explained_variance   | -0.0769     |
|    learning_rate        | 0.00018     |
|    loss                 | 1.94        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0328     |
|    value_loss           | 13.9        |
---

...

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 48.59469696969697
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 22.2         |
| time/                   |              |
|    fps                  | 242          |
|    iterations           | 66           |
|    time_elapsed         | 278          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0038122053 |
|    clip_fraction        | 0.0605       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.14        |
|    explained_variance   | 0.876        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.716        |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0212      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 50.63716216216216
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 26.2         |
| time/                   |              |
|    fps                  | 243          |
|    iterations           | 74           |
|    time_elapsed         | 311          |
|    total_timesteps      | 75776        |
| train/                  |              |
|    approx_kl            | 0.0043813046 |
|    clip_fraction        | 0.0787       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.99        |
|    explained_variance   | 0.888        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.497        |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.0223      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 52.36585365853659
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 29          |
| time/                   |             |
|    fps                  | 246         |
|    iterations           | 82          |
|    time_elapsed         | 340         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.005050987 |
|    clip_fraction        | 0.0925      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.98       |
|    explained_variance   | 0.918       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.407       |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.022      |
|    value_loss           | 1.26

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 53.836666666666666
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 31.2         |
| time/                   |              |
|    fps                  | 250          |
|    iterations           | 90           |
|    time_elapsed         | 368          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0049069067 |
|    clip_fraction        | 0.0917       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.98        |
|    explained_variance   | 0.901        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.568        |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0263      |
|    value_l

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 55.09591836734694
All assignments history: []
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | 32.4       |
| time/                   |            |
|    fps                  | 255        |
|    iterations           | 98         |
|    time_elapsed         | 392        |
|    total_timesteps      | 100352     |
| train/                  |            |
|    approx_kl            | 0.00718562 |
|    clip_fraction        | 0.136      |
|    clip_range           | 0.15       |
|    entropy_loss         | -1.91      |
|    explained_variance   | 0.934      |
|    learning_rate        | 0.00018    |
|    loss                 | 1          |
|    n_updates            | 970        |
|    policy_gradient_loss | -0.0276    |
|    value_loss           | 0.916      |
----------

In [8]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -86.0
Standard deviation of reward: 0.0
Average successful assignments: 5.9
All assignments history: [5, 3, 8, 3, 5, 6, 5, 4, 4, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -90.4    |
| time/              |          |
|    fps             | 436      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -74.0
Standard deviation of reward: 0.0
Average successful assignments: 7.225
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -91.1        |
| time/                   |              |
|    fps                  | 397          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0063776365 |
|    clip_fraction        | 0.0612       |
|    clip_range           | 0.15         |
|    entropy_loss         | -5.02        |
|    explained_variance   | 0.237        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.92         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0336      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 46.43333333333333
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 22.6        |
| time/                   |             |
|    fps                  | 370         |
|    iterations           | 66          |
|    time_elapsed         | 182         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.004237742 |
|    clip_fraction        | 0.0893      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.95       |
|    explained_variance   | 0.912       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.502       |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.0248     |
|    value_loss           | 1.41

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 48.78243243243243
All assignments history: []
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | 28.3       |
| time/                   |            |
|    fps                  | 372        |
|    iterations           | 74         |
|    time_elapsed         | 203        |
|    total_timesteps      | 75776      |
| train/                  |            |
|    approx_kl            | 0.00470504 |
|    clip_fraction        | 0.0924     |
|    clip_range           | 0.15       |
|    entropy_loss         | -1.89      |
|    explained_variance   | 0.931      |
|    learning_rate        | 0.00018    |
|    loss                 | 0.228      |
|    n_updates            | 730        |
|    policy_gradient_loss | -0.025     |
|    value_loss           | 1.07       |
----------

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 50.73719512195122
All assignments history: []
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | 31.1       |
| time/                   |            |
|    fps                  | 371        |
|    iterations           | 82         |
|    time_elapsed         | 226        |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.00522885 |
|    clip_fraction        | 0.0979     |
|    clip_range           | 0.15       |
|    entropy_loss         | -1.86      |
|    explained_variance   | 0.941      |
|    learning_rate        | 0.00018    |
|    loss                 | 0.786      |
|    n_updates            | 810        |
|    policy_gradient_loss | -0.027     |
|    value_loss           | 0.884      |
----------

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 52.388888888888886
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 33           |
| time/                   |              |
|    fps                  | 370          |
|    iterations           | 90           |
|    time_elapsed         | 248          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0050236816 |
|    clip_fraction        | 0.0951       |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.86        |
|    explained_variance   | 0.951        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.155        |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0241      |
|    value_l

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 53.78928571428571
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 34.1        |
| time/                   |             |
|    fps                  | 370         |
|    iterations           | 98          |
|    time_elapsed         | 271         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.006544858 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.78       |
|    explained_variance   | 0.957       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.0173      |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0275     |
|    value_loss           | 0.61

In [9]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -92.0
Standard deviation of reward: 0.0
Average successful assignments: 4.05
All assignments history: [5, 5, 5, 5, 5, 1, 4, 3, 6, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -91.8    |
| time/              |          |
|    fps             | 424      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -64.0
Standard deviation of reward: 0.0
Average successful assignments: 7.575
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -91.7       |
| time/                   |             |
|    fps                  | 401         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.006361192 |
|    clip_fraction        | 0.071       |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.02       |
|    explained_variance   | -0.127      |
|    learning_rate        | 0.00018     |
|    loss                 | 2.26        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0326     |
|    value_loss           | 14.8        |
-

...

-------- Rollout Summary --------
Total mean reward: 32.0
Standard deviation of reward: 0.0
Average successful assignments: 46.403030303030306
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 15.9        |
| time/                   |             |
|    fps                  | 363         |
|    iterations           | 66          |
|    time_elapsed         | 185         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.003861435 |
|    clip_fraction        | 0.0768      |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.14       |
|    explained_variance   | 0.927       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.265       |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.0252     |
|    value_loss           | 1.1

-------- Rollout Summary --------
Total mean reward: 34.0
Standard deviation of reward: 0.0
Average successful assignments: 48.18783783783784
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 18.6         |
| time/                   |              |
|    fps                  | 363          |
|    iterations           | 74           |
|    time_elapsed         | 208          |
|    total_timesteps      | 75776        |
| train/                  |              |
|    approx_kl            | 0.0045748586 |
|    clip_fraction        | 0.0998       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.1         |
|    explained_variance   | 0.927        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.962        |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.0297      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 36.0
Standard deviation of reward: 0.0
Average successful assignments: 49.75792682926829
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 21.1         |
| time/                   |              |
|    fps                  | 363          |
|    iterations           | 82           |
|    time_elapsed         | 230          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0055428557 |
|    clip_fraction        | 0.11         |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.07        |
|    explained_variance   | 0.925        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.202        |
|    n_updates            | 810          |
|    policy_gradient_loss | -0.0316      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 38.0
Standard deviation of reward: 0.0
Average successful assignments: 51.10611111111111
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 23.1         |
| time/                   |              |
|    fps                  | 363          |
|    iterations           | 90           |
|    time_elapsed         | 253          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0049646497 |
|    clip_fraction        | 0.0958       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.05        |
|    explained_variance   | 0.933        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.18         |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0273      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 52.30408163265306
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 24.7        |
| time/                   |             |
|    fps                  | 363         |
|    iterations           | 98          |
|    time_elapsed         | 276         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.006441596 |
|    clip_fraction        | 0.0909      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.99       |
|    explained_variance   | 0.952       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.0948      |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0254     |
|    value_loss           | 0.66

In [10]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -100.0
Standard deviation of reward: 0.0
Average successful assignments: 2.3
All assignments history: [6, 7, 2, 4, 0, 7, 4, 5, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -90.8    |
| time/              |          |
|    fps             | 383      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -60.0
Standard deviation of reward: 0.0
Average successful assignments: 7.225
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -91.1        |
| time/                   |              |
|    fps                  | 376          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0071851453 |
|    clip_fraction        | 0.0924       |
|    clip_range           | 0.15         |
|    entropy_loss         | -5.02        |
|    explained_variance   | -0.0624      |
|    learning_rate        | 0.00018      |
|    loss                 | 2.92         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0357      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 47.88333333333333
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 22.7         |
| time/                   |              |
|    fps                  | 368          |
|    iterations           | 66           |
|    time_elapsed         | 183          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0055714166 |
|    clip_fraction        | 0.0928       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.09        |
|    explained_variance   | 0.93         |
|    learning_rate        | 0.00018      |
|    loss                 | 1.05         |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0242      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 50.020270270270274
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 27.8        |
| time/                   |             |
|    fps                  | 369         |
|    iterations           | 74          |
|    time_elapsed         | 205         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.005118493 |
|    clip_fraction        | 0.0874      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.95       |
|    explained_variance   | 0.932       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.209       |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0275     |
|    value_loss           | 1.1

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 51.80853658536585
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 30.9        |
| time/                   |             |
|    fps                  | 368         |
|    iterations           | 82          |
|    time_elapsed         | 227         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.004824507 |
|    clip_fraction        | 0.0957      |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.88       |
|    explained_variance   | 0.951       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.169       |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.026      |
|    value_loss           | 0.87

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 53.32666666666667
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 33          |
| time/                   |             |
|    fps                  | 368         |
|    iterations           | 90          |
|    time_elapsed         | 249         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.005843224 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.85       |
|    explained_variance   | 0.956       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.174       |
|    n_updates            | 890         |
|    policy_gradient_loss | -0.0318     |
|    value_loss           | 0.80

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 54.61887755102041
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 33.9        |
| time/                   |             |
|    fps                  | 368         |
|    iterations           | 98          |
|    time_elapsed         | 272         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.005522723 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.88       |
|    explained_variance   | 0.947       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.108       |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0288     |
|    value_loss           | 0.84

In [11]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks100.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


Using cpu device


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-------- Rollout Summary --------
Total mean reward: -96.0
Standard deviation of reward: 0.0
Average successful assignments: 3.2
All assignments history: [4, 1, 5, 4, 6, 9, 4, 2, 5, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -91.2    |
| time/              |          |
|    fps             | 471      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -74.0
Standard deviation of reward: 0.0
Average successful assignments: 5.875
All assignments history: []
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | -91.5      |
| time/                   |            |
|    fps                  | 413        |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 2048       |
| train/                  |            |
|    approx_kl            | 0.00621165 |
|    clip_fraction        | 0.0601     |
|    clip_range           | 0.15       |
|    entropy_loss         | -5.02      |
|    explained_variance   | -0.0863    |
|    learning_rate        | 0.00018    |
|    loss                 | 2.29       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0332    |
|    value_loss           | 15.4       |
---------------------

...

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 45.3
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 18.3         |
| time/                   |              |
|    fps                  | 368          |
|    iterations           | 66           |
|    time_elapsed         | 183          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0045389794 |
|    clip_fraction        | 0.076        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.21        |
|    explained_variance   | 0.858        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.24         |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0233      |
|    value_loss           

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 47.5722972972973
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 24.4         |
| time/                   |              |
|    fps                  | 367          |
|    iterations           | 74           |
|    time_elapsed         | 205          |
|    total_timesteps      | 75776        |
| train/                  |              |
|    approx_kl            | 0.0050403783 |
|    clip_fraction        | 0.0902       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.1         |
|    explained_variance   | 0.893        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.537        |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.0235      |
|    value_los

-------- Rollout Summary --------
Total mean reward: 40.0
Standard deviation of reward: 0.0
Average successful assignments: 49.447560975609754
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 26.2         |
| time/                   |              |
|    fps                  | 367          |
|    iterations           | 82           |
|    time_elapsed         | 228          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0054631755 |
|    clip_fraction        | 0.107        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.12        |
|    explained_variance   | 0.888        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.16         |
|    n_updates            | 810          |
|    policy_gradient_loss | -0.0283      |
|    value_l

-------- Rollout Summary --------
Total mean reward: 42.0
Standard deviation of reward: 0.0
Average successful assignments: 51.05222222222222
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 27.5         |
| time/                   |              |
|    fps                  | 366          |
|    iterations           | 90           |
|    time_elapsed         | 251          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0050321273 |
|    clip_fraction        | 0.0896       |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.04        |
|    explained_variance   | 0.9          |
|    learning_rate        | 0.00018      |
|    loss                 | 0.408        |
|    n_updates            | 890          |
|    policy_gradient_loss | -0.0262      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 44.0
Standard deviation of reward: 0.0
Average successful assignments: 52.425
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 28.9        |
| time/                   |             |
|    fps                  | 365         |
|    iterations           | 98          |
|    time_elapsed         | 274         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.005855756 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.01       |
|    explained_variance   | 0.912       |
|    learning_rate        | 0.00018     |
|    loss                 | 1.01        |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0275     |
|    value_loss           | 1.44        |
-